In [308]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import math

In [416]:
tickers = ['VTI', 'AGG', 'DBC', '^VIX']
data = yf.download(tickers, start="2011-01-01", end="2020-04-30", interval="1d")['Adj Close']
data_na = data.dropna(axis = 0)
data_na.dropna(axis=0, inplace=True)
data_na.reset_index(inplace=True)
data_na['Date'] = data_na['Date'].dt.date


[*********************100%***********************]  4 of 4 completed


In [417]:
data_na['Date'] = pd.to_datetime(data_na['Date'])


In [418]:
# Allocation initiale
initial_allocation = [0.50, 0.10, 0.20, 0.20]  # Allocations initiales pour VTI, AGG, DBC, VIX
portfolio_value = 1


In [419]:
# Créer un DataFrame pour les allocations initiales
allocation_df = pd.DataFrame(index=data_na.index, columns=[f'{col}_alloc' for col in data_na.columns[1:]])

# Initialiser les allocations du portefeuille pour chaque actif
initial_prices = data_na.iloc[0, 1:]  # Prix initiaux des actifs
allocation_df.iloc[0] = [portfolio_value * alloc / price for alloc, price in zip(initial_allocation, initial_prices)]


In [399]:
def rebalance_portfolio(row, prev_alloc, prices, initial_allocation):
    total_value = (prev_alloc * prices).sum()  # Calcul de la valeur totale du portefeuille
    new_alloc = [(total_value * alloc) / price for alloc, price in zip(initial_allocation, prices)]
    return pd.Series(new_alloc)


In [420]:
# DataFrame pour stocker les valeurs du portefeuille
portfolio_df = pd.DataFrame(index=data_na.index, columns=['Date', 'portfolio_value'])
portfolio_df['Date'] = data_na['Date']

In [421]:
# Parcourir chaque année et réallouer le portefeuille au 1er janvier de chaque année
for i in range(1, len(data_na)):
    current_year = pd.to_datetime(data_na.loc[i, 'Date']).year
    previous_year = pd.to_datetime(data_na.loc[i-1, 'Date']).year
    
    if current_year != previous_year:  # Réallocation si on change d'année
        previous_alloc = allocation_df.iloc[i-1].values.astype(float)
        current_prices = data_na.iloc[i, 1:].values.astype(float)
        
        # Réallocation basée sur les prix actuels et les allocations initiales
        allocation_df.iloc[i] = rebalance_portfolio(data_na.iloc[i],previous_alloc, current_prices, initial_allocation)
    else:
        allocation_df.iloc[i] = allocation_df.iloc[i-1]  # Conserver la même allocation
    
    # Utilisation de loc pour éviter les avertissements
    portfolio_df.loc[i, 'portfolio_value'] = (allocation_df.iloc[i] * data_na.iloc[i, 1:].values).sum()


In [423]:
# Initialisation du DataFrame des poids
weights_df = pd.DataFrame(index=data_na.index, columns=data_na.columns[1:])

# Poids fixes pour 2011
weights_2011 = initial_allocation  

# Assignation des poids pour 2011
for i in range(len(data_na)):
    current_year = pd.to_datetime(data_na.loc[i, 'Date']).year
    
    if current_year == 2011:
        weights_df.loc[i] = weights_2011  
    elif current_year > 2011:
        # Si c'est la première occurrence de l'année
        if i == 0 or pd.to_datetime(data_na.loc[i - 1, 'Date']).year != current_year:
            # Récupérer les prix de la première ligne de l'année
            first_row_prices = data_na.iloc[i, 1:].values.astype(float)
            # Calculer les poids basés sur la première ligne de l'année
            weights_df.loc[i] = fixed_weights = first_row_prices / first_row_prices.sum()
        else:
            # Pour les autres jours de la même année, conserver les mêmes poids
            weights_df.loc[i] = weights_df.loc[i - 1]



In [424]:
# Supprimer la dernière ligne du DataFrame des poids
weights_df = weights_df.drop(weights_df.index[-1]).reset_index(drop=True)


In [425]:
# Calculer les rendements quotidiens
returns_df = data_na.iloc[:, 1:].pct_change().fillna(0)
returns_df['Date'] = data_na['Date']
returns_df = returns_df[['Date'] + list(returns_df.columns[:-1])]



In [426]:
returns_df = returns_df.drop(index=returns_df.index[0]).reset_index(drop=True)

In [427]:
returns_df = returns_df.drop(columns=['Date'])

In [428]:
# Calculer le produit terme à terme
weighted_returns = returns_df.values * weights_df.values

# Somme sur les lignes pour obtenir le rendement du portefeuille
portfolio_returns = weighted_returns.sum(axis=1)

In [429]:
portfolio_returns

array([-0.004203117019554437, -0.004394377247478809, 0.002763454517601349,
       ..., 0.003785344700315989, 0.00011441350426329898,
       0.01360897261630044], dtype=object)

In [430]:
# Calcul de l'espérance des rendements (moyenne)
expected_return = portfolio_returns.mean()
expected_return = (expected_return+1)**252-1

# Calcul de la standard déviation
std_dev = portfolio_returns.std()
std_dev = std_dev * math.sqrt(252)
risk_free_rate = 0.0

# Calcul du Sharpe ratio
sharpe_ratio = (expected_return - risk_free_rate) / std_dev

# Affichage des résultats
print(f"Espérance des rendements : {expected_return:.4f}")
print(f"Standard déviation : {std_dev:.4f}")
print(f"Sharpe ratio : {sharpe_ratio:.4f}")


Espérance des rendements : 0.1335
Standard déviation : 0.1134
Sharpe ratio : 1.1771


In [327]:
# Initialisation des listes pour stocker les premières données de chaque année
weights_first_list = []
prices_first_list = []
returns_first_list = []
allocation_first_list = []

# Boucle sur les années de 2011 à 2020
for year in range(2011, 2021):
    # Filtrer les données pour l'année en question
    mask = pd.to_datetime(data_na['Date']).dt.year == year
    
    # Stocker la première ligne de chaque année dans les listes
    weights_first_list.append(weights_df.loc[mask].iloc[0])
    prices_first_list.append(data_na.loc[mask].iloc[0])
    returns_first_list.append(returns_df.loc[mask].iloc[0])
    allocation_first_list.append(allocation_df.loc[mask].iloc[0])

# Convertir les listes en DataFrames
weights_first_df = pd.DataFrame(weights_first_list).reset_index(drop=True)
prices_first_df = pd.DataFrame(prices_first_list).reset_index(drop=True)
returns_first_df = pd.DataFrame(returns_first_list).reset_index(drop=True)
allocation_first_df = pd.DataFrame(allocation_first_list).reset_index(drop=True)


## Prise en compte du cout de transaction

In [431]:
C = 0.01  

# Initialisation du vecteur des coûts de transaction
transaction_costs = np.zeros(len(weights_df))

# Calcul des coûts de transaction
for t in range(2, len(weights_df)):
    # Calcul des différences de poids
    diff_weights = weights_df.iloc[t - 1].values - weights_df.iloc[t - 2].values
    
    # Somme des valeurs absolues des différences
    sum_abs_diff = np.sum(np.abs(diff_weights))
    
    # Coût de transaction pour le temps t
    transaction_costs[t] = C * sum_abs_diff



In [436]:
portfolio_with_transac_cost = portfolio_returns +transaction_costs

In [437]:
# Calcul de l'espérance des rendements (moyenne)
expected_return = portfolio_with_transac_cost.mean()
expected_return = (expected_return+1)**252-1

# Calcul de la standard déviation
std_dev = portfolio_with_transac_cost.std()
std_dev = std_dev * math.sqrt(252)
risk_free_rate = 0.0

# Calcul du Sharpe ratio
sharpe_ratio = (expected_return - risk_free_rate) / std_dev

# Affichage des résultats
print(f"Espérance des rendements : {expected_return:.4f}")
print(f"Standard déviation : {std_dev:.4f}")
print(f"Sharpe ratio : {sharpe_ratio:.4f}")


Espérance des rendements : 0.1347
Standard déviation : 0.1134
Sharpe ratio : 1.1885


## Prise en compte de la volatilité scaling

In [442]:
# Initialisation du DataFrame pour stocker les volatilités ex-ante
volatility_df = pd.DataFrame(index=returns_df.index, columns=returns_df.columns)

# Boucle sur chaque colonne (actif) pour calculer la volatilité
for col in returns_df.columns:
    # Calcul de la variance exponentiellement pondérée sur 50 jours
    ew_var = returns_df[col].ewm(span=50, adjust=False).var()
    
    # Calcul de la volatilité ex-ante en prenant la racine carrée de la variance
    volatility_df[col] = ew_var.shift(1).apply(lambda x: x**0.5)



In [439]:
volatility_df

Ticker,AGG,DBC,VTI,^VIX
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,0.003615,0.016967,0.005289,0.005411
3,0.002556,0.012003,0.003751,0.018595
4,0.002476,0.010481,0.003070,0.015225
...,...,...,...,...
2340,0.009805,0.020153,0.038064,0.125889
2341,0.009615,0.019757,0.037311,0.123493
2342,0.009429,0.019436,0.036679,0.124184
2343,0.009304,0.019245,0.036098,0.122673


In [450]:
# Définition du paramètre alpha
vol_target = 0.1  

vol_scale_df = vol_target / volatility_df


In [451]:
vol_scale_df

Ticker,AGG,DBC,VTI,^VIX
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,27.661773,5.893858,18.906104,18.480298
3,39.126708,8.331284,26.661787,5.377897
4,40.389192,9.540813,32.573078,6.568140
...,...,...,...,...
2340,10.199390,4.962047,2.627130,0.794350
2341,10.400435,5.061499,2.680198,0.809760
2342,10.605826,5.145042,2.726392,0.805257
2343,10.747950,5.196251,2.770257,0.815172
